# Non-independence.

Let $A \in \{0, 1\}^{n \times n}$ be an $n \times n$ matrix with each entry sampled i.i.d. from $\{0, 1\}$, and let $b \in \{0, 1\}^n$ be a column vector of length $n$ again with each entry sampled i.i.d. from $\{0, 1\}$.

Clearly each entry of $A b$ is binomially distributed as $B(n, 1/4)$, but how are they jointly distributed?
Are the entries of $A b$ mutually independent, or even pairwise independent? We answer both in the negative, but show that in the $n = 2$ case that the entries are mutually independent conditioned on $b$, and assert generality without proof.

In [1]:
import itertools, collections, numpy

# This dictionary will accumulate our occurence counts for each possible value of $A b$.
outcomes = collections.defaultdict(lambda: 0)

# Let's test all 2^6 possible choices of bits that comprise $A$ and $b$.
for bits in itertools.product(*[(0, 1)]*6):
    # Take the first four bits and make a 2x2 matrix out of them.
    A = numpy.array(bits[:4]).reshape((2, 2))
    # Take the last two bits as a vector.
    b = bits[4:]
    # Take the matrix product, and increment the corresponding bucket in `outcomes`.
    outcome = A.dot(b)
    outcomes[tuple(outcome)] += 1

outcomes

defaultdict(<function __main__.<lambda>>,
            {(0, 0): 25,
             (0, 1): 10,
             (0, 2): 1,
             (1, 0): 10,
             (1, 1): 12,
             (1, 2): 2,
             (2, 0): 1,
             (2, 1): 2,
             (2, 2): 1})

In [2]:
# Convert our above table of occurences into a 3x3 matrix of occurence counts.
J = numpy.array([
    outcomes[i, j]
    for i in xrange(3)
    for j in xrange(3)
]).reshape((3, 3))
J

array([[25, 10,  1],
       [10, 12,  2],
       [ 1,  2,  1]])

We have now computed a matrix $J$ that captures the joint distribution of the two entries of $A b$, satisfying the following by construction:

$$ \mathbb{P}[ A b = (x, y) ] \propto J_{x, y} $$

The definition of the first entry of $A b$ being independent from the second entry of $A b$ is that:

$$ \mathbb{P}[ A b = (x, y) ] = \mathbb{P}[ (A b)_1 = x ] \cdot \mathbb{P}[ (A b)_2 = y ] $$

Substituting in our $J$ from above, we see that this independence would imply that:

$$ J_{x, y} \propto \underbrace{\mathbb{P}[ (A b)_1 = x ]}_{f(x)} \cdot \underbrace{\mathbb{P}[ (A b)_2 = y ]}_{g(y)} $$

Here we have named the marginal distributions of the two entries of $A b$ as $f$ and $g$ respectively.

This condition is simply separability, and would allow us to rewrite $J$ as an outer-product of the two marginal distribution vectors:

$$ J_{x, y} \propto [f(0), f(1), f(2)]^T [g(0), g(1), g(2)] $$

This would imply that $J$ has rank one, and thus determinant zero.

In [3]:
numpy.linalg.det(J)

127.99999999999997

But lo and behold, $J$ has nonzero determinant, and thus is not of rank one, and therefore is not separable, proving that the two entries $(A b)_1$ and $(A b)_2$ are not independent.

## Conditional independence.

But are $(A b)_1$ and $(A b)_2$ independent conditioned on $b$?
Let's show numerically that they are in the $n = 2$ case.

In [4]:
for b in itertools.product(*[(0, 1)]*2):
    outcomes = collections.defaultdict(lambda: 0)
    for A_bits in itertools.product(*[(0, 1)]*4):
        A = numpy.array(A_bits[:4]).reshape((2, 2))
        outcome = A.dot(b)
        outcomes[tuple(outcome)] += 1
    J = numpy.array([
        outcomes[i, j]
        for i in xrange(3)
        for j in xrange(3)
    ]).reshape((3, 3))
    print "b =", b
    print "J:"
    print J
    print "J's rank:", numpy.linalg.matrix_rank(J)
    print

b = (0, 0)
J:
[[16  0  0]
 [ 0  0  0]
 [ 0  0  0]]
J's rank: 1

b = (0, 1)
J:
[[4 4 0]
 [4 4 0]
 [0 0 0]]
J's rank: 1

b = (1, 0)
J:
[[4 4 0]
 [4 4 0]
 [0 0 0]]
J's rank: 1

b = (1, 1)
J:
[[1 2 1]
 [2 4 2]
 [1 2 1]]
J's rank: 1



Indeed we see that the matrix $J$ is of rank one for each value of $b$, and thus separable, proving the *conditional* independence of $(A b)_1$ and $(A b)_2$ given $b$.